In [1]:
import json
import logging
import numpy as np
import os
import pandas as pd
import re
import sys
import janitor

import tkinter as tk
from datetime import datetime
from IPython.display import clear_output
from pathlib import Path
from tkinter import filedialog
from tkinter import scrolledtext


CONFIG_FILENAME = "config.json"
CONFIG_PATH = os.getcwd()
CONFIG_FILE = os.path.join(CONFIG_PATH, CONFIG_FILENAME)

VARIABLES = [
    'quantity_units', 
    'analyte_name', 
    'data_set', 
    'sample_type',
    'rt_min',
    'm_z_expected',
    'area_of_pi'
]


def get_files(dir_name, file_type):
    list_of_files = sorted(os.listdir(dir_name))
    all_files = list()
    for file in list_of_files:
        full_path = os.path.join(dir_name, file)
        if os.path.isdir(full_path):
            all_files = all_files + get_files(full_path, file_type)
        else:
            if file.endswith(file_type):
                all_files.append(full_path)
    return all_files


def open_dir(current_dir):
    return filedialog.askdirectory(initialdir=current_dir, title="Please select a directory")


def get_home():
    return str(Path.home())


def get_config():
    config_json = None
    if os.path.isfile(CONFIG_FILE):
        with open(CONFIG_FILE) as f:
            config_json = json.load(f)
    else:
        with open(CONFIG_FILE, "w+") as f:
            config_json = {"cwd": get_home()}
            json.dump(config_json, f, indent=4)
    return config_json


def set_config(key, value):
    with open(CONFIG_FILE) as f:
        config = json.load(f)
    with open(CONFIG_FILE, "w") as f:
        config[key] = value
        json.dump(config, f, indent=4)


#def clean_names(df):
    #df.columns = ["_".join(re.findall("[a-z]+", re.sub('(?<!^)(?=[A-Z])', '_', v).lower())) for v in df.columns.to_list()]
    #return df


class DataFile:
    @classmethod
    def read(cls, file, file_type):
        f = "read_" + file_type
        # for invalid file type, fn is the lambda function that returns error message
        fn = getattr(cls, f, lambda: "Invalid file type")
        return fn(file)

    def read_csv(file):
        return pd.read_csv(file)
    
    def read_xlsx(file):
        return pd.read_excel(file)


class Application(tk.Frame):
    def __init__(self, master=None):
        tk.Frame.__init__(self, master)
        self.pack(fill='both')
        self.master.title('ANPC - Long2Wide')
        self.file_type = tk.StringVar(value=".xlsx")
        self.load_config()
        self.cwd_label_text = ""
        self.create_widgets()
        self.df = pd.DataFrame()

    def load_config(self):
        self.config = get_config()

    def get_current_dir(self):
        return self.config["cwd"]

    def process_data(self):
        current_dir = self.get_current_dir()
        file_type = self.file_type.get()
        files = get_files(current_dir, file_type)
        
        file_type = file_type[1:]
        if len(files):
            data_file = DataFile()
            for file in files:
                """
                Original R code:
                df = read_excel(path  = paste(filelist[idx_F]),col_names = T)
                df = clean_names(df)
                df_red = select(df, 'quantity_units', 'analyte_name', 'data_set', 'sample_type', 'rt_min', 'm_z_expected', 'area_of_pi')
                df_red$quantity_units=as.numeric(df_red$quantity_units)
                dm=dcast(data=df_red, formula=data_set+sample_type~analyte_name, value.var='quantity_units') # fun.aggregate=mean)
                dm_aa_all = rbind(dm_aa_all,dm)
                """
                df = data_file.read(file, file_type)
                df = janitor.clean_names(df, remove_special=True, case_type='snake')
                df = df[VARIABLES]
                df['quantity_units'] = pd.to_numeric(df['quantity_units'], errors='coerce')
                df = df.pivot_table(index=['data_set', 'sample_type'], columns='analyte_name', values='quantity_units')  # , aggfunc=np.mean)
                self.df = self.df.append(df)

    def long_to_wide(self):
        self.process_data()

        current_dir = self.get_current_dir()
        today = datetime.today()
        timestamp = f"{today.year}{today.month:02}{today.day:02}_{today.hour:02}{today.minute:02}{today.second:02}"
        filename = f"Flipped_{timestamp}.csv"
        report_path = os.path.join(current_dir, filename)
        self.df.to_csv(report_path)
        self.process_message.configure(text=f"Saved {report_path}")

    def select_cwd(self):
        old = self.config["cwd"]
        new = open_dir(old)
        if new:
            set_config("cwd", new)
            self.config["cwd"] = new
            self.cwd_label_text = new
            if old != new:
                self.cwd_label.configure(text="Changed", fg="green")
                self.dir_lf.configure(text=new)
            else:
                self.cwd_label.configure(text="Not changed", fg="blue")

    def add_cwd_selector(self):
        cwd = self.config["cwd"]
        self.dir_lf = tk.LabelFrame(self.cwd_lf, text=cwd, padx=2, pady=2, relief=tk.FLAT, bg="#e6f3ff")
        self.dir_lf.pack(side=tk.LEFT, padx=2, pady=2)
        cwd_button = tk.Button(self.dir_lf, text="Change CWD", command=self.select_cwd)
        cwd_button.pack(side=tk.LEFT, padx=2, pady=2)
        self.cwd_label = tk.Label(self.dir_lf, text=self.cwd_label_text, fg="#000", bg="#e6f3ff")
        self.cwd_label.pack(side=tk.LEFT, padx=2, pady=2)

        type_lf = tk.LabelFrame(self.cwd_lf, text="File Type", padx=2, pady=2, relief=tk.FLAT, bg="#e6f3ff")
        type_lf.pack(side=tk.LEFT, padx=2, pady=2)
        types = [
            (".xlsx", ".xlsx"),
            (".csv", ".csv"),
        ]                
        
        for text, ftype in types:
            rb_ftype = tk.Radiobutton(type_lf, text=text, variable=self.file_type, value=ftype)
            rb_ftype.pack(side=tk.LEFT, padx=2, pady=5)

    def add_process_controls(self):
        process_button = tk.Button(self.process_lf, text="Flip (Long to Wide)", command=self.long_to_wide)
        process_button.pack(side=tk.LEFT, padx=2, pady=2)
        self.process_message = tk.Label(self.process_lf, text=None, fg="#000", bg="#e6f3ff")
        self.process_message.pack(side=tk.LEFT, padx=2, pady=2)

    def add_controls(self):
        self.add_cwd_selector()
        self.add_process_controls()

    def exit(self):
        clear_output()
        self.master.destroy()

    def hide_me(self, widget):
        widget.pack_forget()

    def show_me(self, widget):
        widget.pack(side=tk.LEFT, padx=2, pady=2)

    def show_cwd_lf(self):
        self.hide_me(self.process_lf)
        self.show_me(self.cwd_lf)
        self.menubar.entryconfig(index=1, background="#e6f3ff")
        self.menubar.entryconfig(index=2, background="#ddd")

    def show_process_lf(self):
        self.hide_me(self.cwd_lf)
        self.show_me(self.process_lf)
        self.menubar.entryconfig(index=1, background="#ddd")
        self.menubar.entryconfig(index=2, background="#e6f3ff")

    def add_options_bar(self):
        self.optionsbar = tk.Frame(self, bd=1, relief=tk.RIDGE, bg="#e6f3ff")
        self.cwd_lf = tk.Frame(self.optionsbar, bg="#e6f3ff", padx=4, pady=2, relief=tk.FLAT)
        self.process_lf = tk.Frame(self.optionsbar, bg="#e6f3ff", padx=4, pady=2, relief=tk.FLAT)
        self.add_controls()
        self.optionsbar.pack(side=tk.TOP, fill=tk.X)

    def create_widgets(self):
        self.menubar = tk.Menu(master=self.master)
        self.master.config(menu=self.menubar)
        self.cwd_command = self.menubar.add_command(label="CWD", command=self.show_cwd_lf, activebackground="#e6f3ff")
        self.process_command = self.menubar.add_command(label="Flip", command=self.show_process_lf, activebackground="#e6f3ff")
        self.exit_command = self.menubar.add_command(label="Exit", command=self.exit, activebackground="#e6f3ff", foreground="red")
        self.add_options_bar()


root = tk.Tk()
app = Application(root)
root.geometry("900x400")
root.mainloop()


In [2]:
#